# Initialize Examples and Experiment Functions

In [4]:
%reload_ext autoreload
%autoreload 2

from boat_models import Boat, TwoInputBoat, ThreeInputBoat
import numpy as np
import pickle
from IPython.display import HTML
from states_initialization import states_initialization
from trajectory_planner import BoatConfigurationPlanning, bold

def anim_plot(split, i):
    print '%f seconds' % time_configurations[split][i-1][-1]
    return HTML(anim_configurations[split][i-1])

def to_x0_xN(xy0, xyN, final_offset):
    num_boats = len(xy0)
    
    x0 = np.zeros((len(xy0), Boat.num_states))
    xN = np.zeros((len(xy0), Boat.num_states))

    x0[:,:2]=xy0
    xN[:,:2]=xyN
    xN[:,:2]+=final_offset
    
    return (x0,xN)      

def experiment(boat, x0, xN, steps=100, dt_init=0.4, expansion_N=None, shapemove_N=None, min_time=5, max_time=100, expansion_factor=1.4, initialize=True, example_name=None, init_name=None):
    boats_S_init, assignments = states_initialization(x0, xN, steps, dt_init, expansion_factor=expansion_factor, expansion_N=expansion_N, shapemove_N=shapemove_N)
    xN_assigned = xN[assignments]
    planner = BoatConfigurationPlanning(boat)
    initialization = boats_S_init if initialize else None
    S, U, time, mp, result, solve_time = planner.compute_trajectory(min_time, max_time, x0, xN_assigned, step_initialization=dt_init, states_initialization=initialization)
    return {
            'S_init':boats_S_init, 'S':S, 'U':U, 'x0':x0, 'xN':xN, 'N':steps, 'time':time, 'result':str(result), 'solve_time':solve_time, 'example_name': example_name, 'init_name':init_name, 'boat_type': boat.__class__.__name__,
            'params':{'expansion_N':expansion_N, 'shapemove_N':shapemove_N, 'expansion_factor':expansion_factor, 'min_time':min_time, 'max_time':max_time, 'dt_init':dt_init}
           }

def run_experiments(boats, examples):
    experiments = {}  
    for boat in boats:
        print bold('====%s EXPERIMENTS====\n' % boat.__class__.__name__)
        boat_experiments = {}
        #print bold('----NO_INIT----\n')
        #boat_experiments['no_init']              = [experiment(boat,x0,xN,initialize=False, example_name=name, init_name='none')                                               for x0,xN,name in examples]
        print bold('----INTERPOLATE----')
        boat_experiments['init']                 = [experiment(boat,x0,xN, example_name=name, init_name='interpolate')                                                         for x0,xN,name in examples]
        print bold('----INTERPOLATE EXPANSION----')
        boat_experiments['init_expansion']       = [experiment(boat,x0,xN, expansion_N=20, example_name=name, init_name='interpolate expansion')                               for x0,xN,name in examples]
        print bold('----INTERPOLATE SHAPE----')
        boat_experiments['init_shape']           = [experiment(boat,x0,xN,shapemove_N=50, example_name=name, init_name='interpolate shape')                                    for x0,xN,name in examples]
        print bold('----INTERPOLATE SHAPE EXPANSION----')
        boat_experiments['init_shape_expansion'] = [experiment(boat,x0,xN,shapemove_N=50, expansion_N=20, example_name=name, init_name='interpolate expansion and shape') for x0,xN,name in examples]
        experiments[boat.__class__.__name__] = boat_experiments
    return experiments

In [5]:
examples = (
        #1
        (((0, 0),),
         ((0, 0),)),

        #2 rotate
        (((0, 0), (0, 1.5)),
         ((1, 0), (2.5, 0))),

        #4 line rotate
        (((2.25, 0), (0.75, 0), (-.75, 0), (-2.25, 0)),
         ((0, 2.25), (0, 0.75), (0, -.75), (0, -2.25))),

        #4 line to 4 grid
        (((2.25, 0), (0.75, 0), (-.75, 0), (-2.25, 0)),
         ((0, 0), (0, 1.5), (1.5, 0), (1.5, 1.5))),

        #4 grid rotate
        (((0, 0), (0, 1.5), (1.5, 0), (1.5, 1.5)),
         ((0, 1.06), (0, -1.06), (-1.06, 0), (1.06, 0))),

        #4 triangle rotate
        (((0, 0), (0, 1.5), (-1.5, 0), (1.5, 0)),
         ((0, 0), (0, 1.5), (0, -1.5), (1.5, 0))),

        #9 triangle to 9 grid
        (((-1.5, -3), (-1.5, -1.5), (-1.5, 0), (-1.5, 1.5), (-1.5, 3), (0, -1.5), (0, 0), (0, 1.5), (1.5, 0)),
         ((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5))),

        #9 line to 9 grid
        (((6,0), (4.5, 0), (3, 0), (1.5, 0), (0, 0), (-1.5, 0), (-3, 0), (-4.5, 0), (-6, 0)),
         ((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5))),

        #9 grid rotate
        (((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5)),
         ((0, 0), (2.12, 0), (-2.12, 0), (1.06, 1.06), (1.06, -1.06), (-1.06, 1.06), (-1.06, -1.06), (0, 2.12), (0, -2.12)))

    )

final_offset = ((4, 0))

example_states = tuple([to_x0_xN(xy0, xyN, final_offset) for (xy0, xyN) in examples])
example_names = ['1 Boat Move', '2 Boat Line Rotate','4 Boat Line Rotate', '4 Boat Line to Grid', '4 Boat Grid Rotate', '4 Boat Triangle Rotate', '9 Boat Triangle to Grid', '9 Boat Line to Grid', '9 Boat Grid Rotate']
examples = [(x0,xN,name) for (x0,xN),name in zip(example_states,example_names)]

assert len(example_names) == len(example_states)

# Run Experiments

In [6]:
boats = [TwoInputBoat(), ThreeInputBoat(), Boat()]
experiments1 = run_experiments(boats, examples[:1])

====TwoInputBoat EXPERIMENTS====

----INTERPOLATE----
1
INITIALIZED 1 TwoInputBoat boats
boats_S:(1, 101, 4), boats_U:(1, 100, 2), time_array:(101,)
Number of decision vars 605
0.004490 seconds
ADDING COSTS
0.000179 seconds
ADDING CONSTRAINTS
False
0.080079 seconds
PLANNING
SolutionResult.kSolutionFound
0.045232 seconds

----INTERPOLATE EXPANSION----
1
INITIALIZED 1 TwoInputBoat boats
boats_S:(1, 101, 4), boats_U:(1, 100, 2), time_array:(101,)
Number of decision vars 605
0.002262 seconds
ADDING COSTS
0.000052 seconds
ADDING CONSTRAINTS
False
0.047631 seconds
PLANNING
SolutionResult.kSolutionFound
0.061989 seconds

----INTERPOLATE SHAPE----
[[-4.  0.]]
1
INITIALIZED 1 TwoInputBoat boats
boats_S:(1, 101, 4), boats_U:(1, 100, 2), time_array:(101,)
Number of decision vars 605
0.002323 seconds
ADDING COSTS
0.000066 seconds
ADDING CONSTRAINTS
False
0.071618 seconds
PLANNING
SolutionResult.kSolutionFound
0.054624 seconds

----INTERPOLATE SHAPE EXPANSION----
[[-4.  0.]]
1
INITIALIZED 1 TwoInpu

In [ ]:
boats = [twoInputBoat(min_interboat_clearance=0.4), threeInputBoat(min_interboat_clearance=0.4), Boat(min_interboat_clearance=0.4)]
experiments1 = run_experiments(boats, examples)

# Save Experiments

In [ ]:
#label = '(init)'
#with open('results/experiments'+label+'.pickle', 'wb') as f:
#    pickle.dump(experiments, f)

In [22]:
#label = '(min_interboat_clearance=0.4)'
#with open('results/experiments'+label+'.pickle', 'wb') as f:
#    pickle.dump(experiments, f)